# Analyzing borrowers’ risk of defaulting

## **Introduction** <a id='intro'></a>
My project is a report for a bank’s loan division. We need to find out if a customer’s marital status, income level, loan purposes, and the number of children have an impact on whether they will default on a loan. The bank already has some data on customers’ creditworthiness.

That report will be considered when building the **credit score** of a potential customer. The **credit score** is used to evaluate the ability of a potential borrower to repay their loan.


### Goal: 
We should answer these questions:
- Is there a connection between having kids and repaying a loan on time?
- Is there a connection between marital status and repaying a loan on time?
- Is there a connection between income level and repaying a loan on time?
- How do different loan purposes affect on-time loan repayment?

### Stages 
Data on customers behavior is stored in the file `/datasets/credit_scoring_eng.csv`. There is no information about the quality of the data, so we need to explore it before testing the hypotheses. 

First, evaluate the quality of the data and see whether its issues are significant. Then, during data preprocessing, try to account for the most critical problems.
 
Project consist of four stages:
 1. Data exploration
 2. Data transformation
 3. Restoring missing values
 4. Categorization of data
 5. Checking the Hypotheses

 
[Back to Contents](#back)

## **Open the data file and have a look at the general information.**

Import `pandas`,read the file `credit_scoring_eng.csv` from the `/datasets/` folder and save it in the `df` variable:

In [133]:
# Loading all the libraries
import pandas as pd
import numpy as np
from nltk.stem import SnowballStemmer

# Load the data
df = pd.read_csv('/datasets/credit_scoring_eng.csv')
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,19351.000000
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,26787.568355
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,16475.450632
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,3306.762000
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,16488.504500
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,23202.870000
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,32549.611000
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,362496.645000


## **Data exploration**

**Description of the data**
- `children` - the number of children in the family
- `days_employed` - work experience in days
- `dob_years` - client's age in years
- `education` - client's education
- `education_id` - education identifier
- `family_status` - marital status
- `family_status_id` - marital status identifier
- `gender` - gender of the client
- `income_type` - type of employment
- `debt` - was there any debt on loan repayment
- `total_income` - monthly income
- `purpose` - the purpose of obtaining a loan


Print the dataset size and first 15 rows:

In [134]:
# Let's see how many rows and columns our dataset has
df.shape


(21525, 12)

In [135]:
# let's print the first 15 rows
df.head(15)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


The dataset contains unrealistic data, implicit duplicates, and missing values. That may affect all future operations with data. All issues should be fixed. 

Obtaining the general information about the table with one command:

In [136]:
# Get info on data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


As we can see above, only two columns contain missing values: `'days_employed'` and `'total_income'`. Also, both columns are float data types. Due to specific data purposes, I assume it should be an integer.

Print the filtered table with missing values for `'days_employed'` column:

In [137]:
# Let's look at the filtered table with missing values in the the first column with missing data

print(df['days_employed'].value_counts(dropna=False))


 NaN            2174
-1645.463049       1
-6620.396473       1
-1238.560080       1
-3047.519891       1
                ... 
-2849.351119       1
-5619.328204       1
-448.829898        1
-1687.038672       1
-206.107342        1
Name: days_employed, Length: 19352, dtype: int64


Print the filtered table with missing values for `'total_income'` column:

In [138]:
print(df['total_income'].value_counts(dropna=False))

NaN          2174
17312.717       2
31791.384       2
42413.096       2
26935.722       1
             ... 
48796.341       1
34774.610       1
15710.698       1
19232.334       1
29653.643       1
Name: total_income, Length: 19349, dtype: int64


Missing values seem symmetric. Even so,  if those groups have unrealistic data (e.g. large numbers in the `'days_employed'` column), then we treat that data as missing values which, probably, may change symmetry. However, symmetry may indicate that our missing values are nonrandom.

In [139]:
# Let's apply multiple conditions for filtering data and look at the number of rows in the filtered table.

print(df[['days_employed', 'total_income']].isna().sum())

days_employed    2174
total_income     2174
dtype: int64


Calculate the percentage of the missing values

In [140]:
df[['days_employed', 'total_income']].isna().sum()/df.shape[0]*100

days_employed    10.099884
total_income     10.099884
dtype: float64

**Intermediate conclusion**

The number of rows in the filtered table matches the number of missing values. That means both those columns have quantitative missing values.

The percentage of the missing values compared to the whole dataset is 10.1%. It is a considerably big piece of data. Due to the quantitive type of missing values, we can't drop them or replace them with 0. That may change distribution.
The best way to deal with missing values is:
- Consider whether the missing data could be due to the specific client characteristic. Decide which characteristic might be the reason. 
- Check whether there's any dependence missing values have on the value of other indicators with the columns with identified specific client characteristic.
- Check distribution and, based on that data, choose the value for replacing missing values.

Let's guess that data was missed due to employment status. 
- First, create a new dataset `'df_missing'` with all rows from the original dataset which contains at least one missed value, and print filtered by employment status table. Remove columns `'days_employed'` and `'total_income'`
- Second, checking distribution (in %). That will show the percentage of missing values for each employment type in the original dataset.


In [141]:
# Let's investigate clients who do not have data on identified characteristic and the column with the missing values

df_missing = df[df.isna().any(axis=1)]
df_missing = df_missing.drop(columns = ['days_employed','total_income'])

print(df_missing['income_type'].value_counts())
print()
print(df['income_type'].value_counts())


employee         1105
business          508
retiree           413
civil servant     147
entrepreneur        1
Name: income_type, dtype: int64

employee                       11119
business                        5085
retiree                         3856
civil servant                   1459
entrepreneur                       2
unemployed                         2
student                            1
paternity / maternity leave        1
Name: income_type, dtype: int64


In [142]:
# Checking distribution
#NaN will show that there is no missing values for that employment type
print(df['income_type'].value_counts() / df.shape[0]*100)
print()
print(df_missing['income_type'].value_counts()/df['income_type'].value_counts()*100)


employee                       51.656214
business                       23.623693
retiree                        17.914053
civil servant                   6.778165
entrepreneur                    0.009292
unemployed                      0.009292
student                         0.004646
paternity / maternity leave     0.004646
Name: income_type, dtype: float64

business                        9.990167
civil servant                  10.075394
employee                        9.937944
entrepreneur                   50.000000
paternity / maternity leave          NaN
retiree                        10.710581
student                              NaN
unemployed                           NaN
Name: income_type, dtype: float64


For employment type which quantity is less than 1% data can be neglected. Therefore, there is no evidence of the relation between missed values and employment type.

**Possible reasons for missing values in data**

Values might be missed randomly due to technical issues or might have a specific pattern. 
There are some examples:
- When the couple takes a loan together that means one person signer and another person co-signer. There is no need to know the co-signer's salary if the signer has enough income to make the bank feel safe. 
- Customer didn't return his loan and hide his income.

Another reason can be the time when data was entered. It is impossible to check due to a lack of data.


Checking the distribution in the whole dataset:

In [143]:
# Checking the distribution in the whole dataset

for col in df_missing.columns:
    print(df[col].value_counts() / df.shape[0]*100)
    print()
    print(df_missing[col].value_counts()/df[col].value_counts()*100)
    print()

 0     65.732869
 1     22.383275
 2      9.547038
 3      1.533101
 20     0.353078
-1      0.218351
 4      0.190476
 5      0.041812
Name: children, dtype: float64

-1      6.382979
 0     10.170330
 1      9.858863
 2      9.927007
 3     10.909091
 4     17.073171
 5     11.111111
 20    11.842105
Name: children, dtype: float64

35    2.866434
40    2.829268
41    2.819977
34    2.801394
38    2.778165
42    2.773519
33    2.699187
39    2.662021
31    2.601626
36    2.578397
44    2.541231
29    2.531940
30    2.508711
48    2.499419
37    2.494774
50    2.387921
43    2.383275
32    2.369338
49    2.360046
28    2.336818
45    2.308943
27    2.290360
56    2.262485
52    2.248548
47    2.229965
54    2.225319
46    2.206736
58    2.141696
57    2.137050
53    2.132404
51    2.081301
59    2.062718
55    2.058072
26    1.895470
60    1.751452
25    1.658537
61    1.649245
62    1.635308
63    1.249710
64    1.231127
24    1.226481
23    1.180023
65    0.901278
66    0.850174
22  

**Conclusions**


There is no patterns in distribution. The missing values are random for columns `'total_income'` and  `'days_employed'`.

Current Data Frame contains a lot of problematic data. Such as:
- Obvious and implicit duplicates
- Different registres
- Incorrect artifacts
- Incorrect data type
- Negative numbers 
- Missing values

All of that types of issues should be fixed.

## **Data transformation**

### Cleaning the `education` column

Checking the data in the `education` column. Remove implicit duplicates and fixing educational information if needed.

In [144]:
# Let's see all values in education column to check if and what spellings will need to be fixed
print(df['education'].sort_values().unique())

["BACHELOR'S DEGREE" "Bachelor's Degree" 'GRADUATE DEGREE'
 'Graduate Degree' 'PRIMARY EDUCATION' 'Primary Education'
 'SECONDARY EDUCATION' 'SOME COLLEGE' 'Secondary Education' 'Some College'
 "bachelor's degree" 'graduate degree' 'primary education'
 'secondary education' 'some college']


In [145]:
# Fix the registers if required
df['education'] =df['education'].str.lower()

In [146]:
# Checking all the values in the column to make sure we fixed them

print(df['education'].sort_values().unique())

["bachelor's degree" 'graduate degree' 'primary education'
 'secondary education' 'some college']


### Cleaning the `children` column

Checking the data in the `children` column:

In [147]:
# Let's see the distribution of values in the `children` column
print(df['children'].value_counts() / df.shape[0]*100)

 0     65.732869
 1     22.383275
 2      9.547038
 3      1.533101
 20     0.353078
-1      0.218351
 4      0.190476
 5      0.041812
Name: children, dtype: float64


In our data frame column `children` numbers, -1 and 20 look unrealistic. The percentage are 0,2 and 0,4,respectively. Incorrect negative values should be replaced with absolute values. 20 should be replaced with the quantile (80% quantile due to large numbers). All values must be natural numbers.

In [148]:
# [fix the data based on your decision]
df['children'] = df['children'].abs()#replace negative numbers
df.loc[df['children'] == 20, 'children'] = df['children'].quantile(0.8)#replace large number with quantile
df['children'] = df['children'].astype('int')#all values should be int

In [149]:
# Checking the `children` column again to make sure it's all fixed

print(df['children'].value_counts())

0    14149
1     4941
2     2055
3      330
4       41
5        9
Name: children, dtype: int64


### Cleaning the `days_employed` column

Checking the data in the `days_employed` column. 
That type of values should be natural number and should not exceed 18250 (50 years).

In [150]:
# Find problematic data in `days_employed`, if they exist, and calculate the percentage

print(df['days_employed'][(df['days_employed'] > 18250) | (df['days_employed'] < 0)].count()/df.shape[0]*100)

89.90011614401858


The amount of problematic data is very high, it could've been due to some technical issues or wrong data entry. `days_employed` contains negative float numbers, and error outliers. Change negative `days_employed` values with absolute values. All data greater than 18250 (50 years) is unrealistic. Those values should be replaced with the quantile (80% quantile because large numbers should be replaced). `days_employed` should contain integer numbers. Due to missing values the best way to change data type after missing values replacing.

In [151]:
# Address the problematic values, if they exist
df['days_employed'] = df['days_employed'].abs()#replace negative values with absolute
df.loc[df['days_employed'] > 18250, 'days_employed'] = df['days_employed'].quantile(q=0.8)#replace large numbers


In [152]:
# Check the result - make sure it's fixed
print(df['days_employed'][(df['days_employed'] > 18250) | (df['days_employed'] < 0)].count())

0


### Cleaning the `dob_years` column

Checking the `dob_years` column. 
Possible issues might be missed values or unreal values. The client's age can't be less than 18 for any type of income type, and the minimum age for retirement income type is 62. 

In [153]:
# Check the `dob_years` for suspicious values and count the percentage

print(df.loc[((df['dob_years'] > 18) & (df['dob_years'] < 62) & (df['income_type'] == 'retiree')) | ((df['dob_years'] < 18)),'dob_years'].count()/df.shape[0]*100)

11.549361207897793


Due to a large number of problematic values, it should be replaced. All values less than 18 should be replaced with the second quantile (50% quantile) based on each type of income.  All values less than 62 with the `retiree` type of income should be replaced with the quantile (80% quantile) based on each type of income. 

In [154]:
# Address the issues in the `dob_years` column, if they exist
def replace_wrong_ages(income_types): #create function
    for income_type in income_types: 
        if income_type =='retiree':#for retiree type of income age can't be less than 62
            df.loc[((df['dob_years'] < 62) & (df['income_type'] == income_type)),'dob_years'] = int(df.loc[((df['dob_years'] > 62) & (df['income_type'] == income_type)),['income_type','dob_years']].quantile(0.8)) 
        else:#for other types of income age can't be less than 18
            df.loc[((df['dob_years'] < 18) & (df['income_type'] == income_type)),'dob_years'] = int(df.loc[((df['dob_years'] > 18) & (df['income_type'] == income_type)),['income_type','dob_years']].quantile(0.5)) 
income_types=['business','civil servant','employee','retiree']
replace_wrong_ages(income_types)#call function
df['dob_years'] = df['dob_years'].astype('int')#all values should be int

In [155]:
# Check the result - make sure it's fixed

print(df.loc[((df['dob_years'] > 18) & (df['dob_years'] < 62) & (df['income_type'] == 'retiree')) | ((df['dob_years'] < 18)),'dob_years'].count())

0


### Cleaning the `family_status` column

Checking the `family_status` column. It is categorical column which may have several problems: spaces, uppercase characters. Remove unnecessary spaces, make all characters lowercase if necessary.

In [156]:
# Let's see the values for the column

print(df['family_status'].value_counts())

married              12380
civil partnership     4177
unmarried             2813
divorced              1195
widow / widower        960
Name: family_status, dtype: int64


In [157]:
# Address the problematic values in `family_status`, if they exist

df['family_status'] = df['family_status'].replace("widow / widower",'widow/widower')

In [158]:
# Check the result - make sure it's fixed

print(df['family_status'].value_counts())

married              12380
civil partnership     4177
unmarried             2813
divorced              1195
widow/widower          960
Name: family_status, dtype: int64


### Cleaning the `gender` column

Check the `gender` column. Missing values should be replaced with the common value.

In [159]:
# Let's see the values in the column

print(df['gender'].value_counts())

F      14236
M       7288
XNA        1
Name: gender, dtype: int64


In [160]:
# Address the problematic values, if they exist

#df['gender'][(df['gender'] == 'XNA')] = df['gender'].mode()
df.loc[df["gender"] == "XNA", "gender"] = 'F'

In [161]:
# Check the result - make sure it's fixed

print(df['gender'].value_counts())

F    14237
M     7288
Name: gender, dtype: int64


### Cleaning the `income_type` column

Check the `income_type` column.  It is categorical column which may have several problems: spaces, uppercase characters. Remove unnecessary spaces, make all characters lowercase if necessary.

In [162]:
# Let's see the values in the column

print(df['income_type'].value_counts())

employee                       11119
business                        5085
retiree                         3856
civil servant                   1459
entrepreneur                       2
unemployed                         2
student                            1
paternity / maternity leave        1
Name: income_type, dtype: int64


In [163]:
# Address the problematic values, if they exist

df['income_type'] = df['income_type'].replace("paternity / maternity leave",'paternity/maternity leave')

In [164]:
# Check the result - make sure it's fixed

print(df['income_type'].value_counts())

employee                     11119
business                      5085
retiree                       3856
civil servant                 1459
entrepreneur                     2
unemployed                       2
student                          1
paternity/maternity leave        1
Name: income_type, dtype: int64


## **Working with missing values**

To speed up working with some data, check processing time for two options: 
- `fillna`-`groupby`-`transform` 
- and `groupby`-`apply`-`fillna`

In [165]:
# Find the optimum decision

#%timeit df['total_income'].fillna(df.groupby(['education','dob_years'])['total_income'].transform('mean'))

#%timeit df.groupby(['education','dob_years'])['total_income'].apply(lambda x: x.fillna(x.mean()))

`fillna`-`groupby`-`transform` option is the best choice.

### Restoring missing values in `total_income`

The best way to replace missing values in `total_income` column is using mean or median value of small groups with unique combination of `age`, `education`, and `income type`. The smaller the groups, the more detail provided, and the harder data is to interpret but it provides more accurate values to replace with.  
First, create new column with assigned age groups. Group people by decade (20s, 30s, etc.). Every group, except the first and last groups, should be the same size. For age data, it is common that older ages be placed in 60+ or 70+ categories. Since the minimum age is 18, we don't need a 0-18 age group. First age group contains clients between 18 and 29. Last age group contains clients 70 and older.

In [166]:
# Let's write a function that calculates the age category

def assign_age_group(age):
    
    if age < 30:
        return '18-29'#1st age group: people 18yo to 29yo
    elif age < 40:
        return '30-39'#2nd age group: people 30yo to 39yo
    elif age < 50:
        return '40-49'#3rd age group: people 40yo to 49yo
    elif age < 60:
        return '50-59'#4th age group: people 50yo to 59yo
    elif age < 70:
        return '60-69'#5th age group: people 60yo to 69yo
    else: 
        return '70+'#6th age group: 70+ seniors    
    

In [167]:
# Test if the function works

#Test function on 26 (should be 18-29)
print(assign_age_group(26))
print()
#Test function on 51 (should be 50-59)
print(assign_age_group(51))
print()
#Test function on 70 (should be 70+)
print(assign_age_group(70))

18-29

50-59

70+


In [168]:
# Creating new column based on function

df['age_group']=df['dob_years'].apply(assign_age_group)

In [169]:
# Checking how values in the new column

df[['dob_years', 'age_group']].head()

,dob_years,age_group
0,42,40-49
1,36,30-39
2,33,30-39
3,32,30-39
4,68,60-69


Income depends on many factors such as:
- Skills
- Education
- Income type 
- Age group
- Common trends

Due to the impossibility and inaccessibility of presenting `common trends` and `skills` in numbers, we have to use `age group`, `education`, and `income type` as factors on which the `total income` values depend.

Create a table that only has data without missing values. This data will be used to restore the missing values

In [170]:
# Create a table without missing values and print a few of its rows to make sure it looks fine

df_ref1 = df.dropna()
df_ref1[['days_employed', 'education', 'income_type','total_income', 'age_group']].head(20)

,days_employed,education,income_type,total_income,age_group
0,8437.673028,bachelor's degree,employee,40620.102,40-49
1,4024.803754,secondary education,employee,17932.802,30-39
2,5623.422610,secondary education,employee,23341.752,30-39
3,4124.747207,secondary education,employee,42820.568,30-39
4,8795.549101,secondary education,retiree,25378.572,60-69
5,926.185831,bachelor's degree,business,40922.170,18-29
6,2879.202052,bachelor's degree,business,38484.156,40-49
7,152.779569,secondary education,employee,21731.829,50-59
8,6929.865299,bachelor's degree,employee,15337.093,30-39
9,2188.756445,secondary education,employee,23108.150,40-49


In [171]:
# Look at the mean values for income based on your identified factors

data_pivot = df_ref1.pivot_table(
    index=['age_group', 'education'],
    columns='income_type',
    values='total_income',
    aggfunc='median',
)
data_pivot

income_type                      business  civil servant    employee  \
age_group education                                                    
18-29     bachelor's degree    28250.2840     24617.5440  24030.5850   
          primary education    24210.1960     30563.3830  24583.5965   
          secondary education  23798.6260     21361.7300  20189.5580   
          some college         25570.8550     21297.7030  21783.0890   
30-39     bachelor's degree    32658.5990     27956.2445  26587.4230   
          graduate degree             NaN     17822.7570  18551.8460   
          primary education    20030.2260     21150.6960  19546.3410   
          secondary education  26292.5770     20870.9730  22079.6140   
          some college         29814.5275     30672.7180  26913.8230   
40-49     bachelor's degree    34592.0450     28425.4810  27427.6640   
          graduate degree             NaN            NaN  31771.3210   
          primary education    27929.0980     78410.7740  21132.3895   
          secondary education  26115.6660     22349.0400  22217.0090   
          some college         35282.0940     26587.6740  26846.2815   
50-59     bachelor's degree    35379.3990     28366.5680  27503.5590   
          graduate degree             NaN            NaN  42945.7940   
          primary education    23066.1310     12709.2755  14599.2580   
          secondary education  24461.0155     21417.6660  21831.2455   
          some college         33073.7630            NaN  22644.3135   
60-69     bachelor's degree    30849.5425     27117.8410  27969.0405   
          graduate degree             NaN            NaN         NaN   
          primary education           NaN            NaN  26188.2420   
          secondary education  27142.1265     22756.9360  21311.4815   
          some college         37146.5350            NaN  32844.1030   
70+       bachelor's degree    36808.9680     57508.0320         NaN   
          primary education           NaN            NaN         NaN   
          secondary education  24259.6870     19530.6775  24660.9010   
          some college                NaN            NaN         NaN   

income_type                    entrepreneur  paternity/maternity leave  \
age_group education                                                      
18-29     bachelor's degree       79866.103                        NaN   
          primary education             NaN                        NaN   
          secondary education           NaN                        NaN   
          some college                  NaN                        NaN   
30-39     bachelor's degree             NaN                        NaN   
          graduate degree               NaN                        NaN   
          primary education             NaN                        NaN   
          secondary education           NaN                   8612.661   
          some college                  NaN                        NaN   
40-49     bachelor's degree             NaN                        NaN   
          graduate degree               NaN                        NaN   
          primary education             NaN                        NaN   
          secondary education           NaN                        NaN   
          some college                  NaN                        NaN   
50-59     bachelor's degree             NaN                        NaN   
          graduate degree               NaN                        NaN   
          primary education             NaN                        NaN   
          secondary education           NaN                        NaN   
          some college                  NaN                        NaN   
60-69     bachelor's degree             NaN                        NaN   
          graduate degree               NaN                        NaN   
          primary education             NaN                        NaN   
          secondary education           NaN                    

In [172]:
# Look at the median values for income based on your identified factors

data_pivot1 = df_ref1.pivot_table(
    index=['age_group', 'education'],
    columns='income_type',
    values='total_income',
    aggfunc='median',
)
data_pivot1

income_type                      business  civil servant    employee  \
age_group education                                                    
18-29     bachelor's degree    28250.2840     24617.5440  24030.5850   
          primary education    24210.1960     30563.3830  24583.5965   
          secondary education  23798.6260     21361.7300  20189.5580   
          some college         25570.8550     21297.7030  21783.0890   
30-39     bachelor's degree    32658.5990     27956.2445  26587.4230   
          graduate degree             NaN     17822.7570  18551.8460   
          primary education    20030.2260     21150.6960  19546.3410   
          secondary education  26292.5770     20870.9730  22079.6140   
          some college         29814.5275     30672.7180  26913.8230   
40-49     bachelor's degree    34592.0450     28425.4810  27427.6640   
          graduate degree             NaN            NaN  31771.3210   
          primary education    27929.0980     78410.7740  21132.3895   
          secondary education  26115.6660     22349.0400  22217.0090   
          some college         35282.0940     26587.6740  26846.2815   
50-59     bachelor's degree    35379.3990     28366.5680  27503.5590   
          graduate degree             NaN            NaN  42945.7940   
          primary education    23066.1310     12709.2755  14599.2580   
          secondary education  24461.0155     21417.6660  21831.2455   
          some college         33073.7630            NaN  22644.3135   
60-69     bachelor's degree    30849.5425     27117.8410  27969.0405   
          graduate degree             NaN            NaN         NaN   
          primary education           NaN            NaN  26188.2420   
          secondary education  27142.1265     22756.9360  21311.4815   
          some college         37146.5350            NaN  32844.1030   
70+       bachelor's degree    36808.9680     57508.0320         NaN   
          primary education           NaN            NaN         NaN   
          secondary education  24259.6870     19530.6775  24660.9010   
          some college                NaN            NaN         NaN   

income_type                    entrepreneur  paternity/maternity leave  \
age_group education                                                      
18-29     bachelor's degree       79866.103                        NaN   
          primary education             NaN                        NaN   
          secondary education           NaN                        NaN   
          some college                  NaN                        NaN   
30-39     bachelor's degree             NaN                        NaN   
          graduate degree               NaN                        NaN   
          primary education             NaN                        NaN   
          secondary education           NaN                   8612.661   
          some college                  NaN                        NaN   
40-49     bachelor's degree             NaN                        NaN   
          graduate degree               NaN                        NaN   
          primary education             NaN                        NaN   
          secondary education           NaN                        NaN   
          some college                  NaN                        NaN   
50-59     bachelor's degree             NaN                        NaN   
          graduate degree               NaN                        NaN   
          primary education             NaN                        NaN   
          secondary education           NaN                        NaN   
          some college                  NaN                        NaN   
60-69     bachelor's degree             NaN                        NaN   
          graduate degree               NaN                        NaN   
          primary education             NaN                        NaN   
          secondary education           NaN                    

The imputation method should be decided after considering the distribution of data: normal distribution and skewed distribution (be it right-skewed or left-skewed). Mean imputation works better if the distribution is normally-distributed or has a Gaussian distribution, while median imputation is preferable for skewed distribution(be it right or left). As we can see in pivot tables mean values are greater than median values. That means the data are skewed to the left, and we have to use the median to replace missing values.

In [173]:
#  Write a function that we will use for filling in missing values

def filling_missing_values(column, factors, function):
    df[column] = df[column].fillna(df.groupby(factors)[column].transform(function))
    return

In [174]:
# Check if it works

filling_missing_values('total_income', ['education','age_group', 'income_type'], 'median')
df[['days_employed', 'education', 'income_type', 'total_income', 'age_group']].head(30)

,days_employed,education,income_type,total_income,age_group
0,8437.673028,bachelor's degree,employee,40620.102,40-49
1,4024.803754,secondary education,employee,17932.802,30-39
2,5623.422610,secondary education,employee,23341.752,30-39
3,4124.747207,secondary education,employee,42820.568,30-39
4,8795.549101,secondary education,retiree,25378.572,60-69
5,926.185831,bachelor's degree,business,40922.170,18-29
6,2879.202052,bachelor's degree,business,38484.156,40-49
7,152.779569,secondary education,employee,21731.829,50-59
8,6929.865299,bachelor's degree,employee,15337.093,30-39
9,2188.756445,secondary education,employee,23108.150,40-49


In [175]:
# Apply it to every row

column='total_income'
factors=['education','age_group', 'income_type']
function= 'median'
filling_missing_values(column, factors, function)

In [176]:
# Check if we got any errors

df[df['total_income'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
1303,1,NaN,70,primary education,3,civil partnership,1,F,employee,0,NaN,transactions with commercial real estate,70+
5936,0,NaN,58,bachelor's degree,0,married,0,M,entrepreneur,0,NaN,buy residential real estate,50-59
8142,0,NaN,64,primary education,3,civil partnership,1,F,civil servant,0,NaN,to have a wedding,60-69


We still have missing values because it was not enougth data for those specific combinations. In pivot table we may notice that we have enough information to find median in `age group` - `education` combination. That data should be used to replase the remaining missing values.

In [177]:
# Replacing missing values if there are any errors

df['total_income'] = df['total_income'].fillna(df.groupby(['education','age_group'])['total_income'].transform('median'))

Obtaining the general information of dataset to check the number of rows in `total_income` column:

In [178]:
# Checking the number of entries in the columns

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
age_group           21525 non-null object
dtypes: float64(2), int64(5), object(6)
memory usage: 2.1+ MB


###  Restoring values in `days_employed`

The amount of emploed days as an income depends on many factors such as:
- Education 
- Age group
- Common trends
- Economic situation in the world or/and country

Due to the impossibility and inaccessibility of presenting `common trends` and `Economic situation` in current Data Frame, we have to use `age group` and `education` as factors on which the `days_employed` values depend.

In [179]:
# Distribution of `days_employed` medians based on your identified parameters

data_pivot1 = df_ref1.pivot_table(
    index=['age_group', 'education'],
    values='days_employed',
    aggfunc='median',
)

data_pivot1

days_employed
age_group education                         
18-29     bachelor's degree      1005.389899
          primary education      1030.219648
          secondary education    1013.920085
          some college            885.268574
30-39     bachelor's degree      1588.713712
          graduate degree        4159.753909
          primary education      1089.565214
          secondary education    1623.096078
          some college           1203.830172
40-49     bachelor's degree      2036.307491
          graduate degree         409.200149
          primary education      1040.381892
          secondary education    2034.017486
          some college           2019.735237
50-59     bachelor's degree      2375.776732
          graduate degree        5352.038180
          primary education      2265.784893
          secondary education    2241.409985
          some college           1898.186430
60-69     bachelor's degree      8795.549101
          graduate degree        8795.549101
          primary education      8795.549101
          secondary education    8795.549101
          some college           8795.549101
70+       bachelor's degree      8795.549101
          primary education      8795.549101
          secondary education    8795.549101
          some college           8795.549101

In [180]:
# Distribution of `days_employed` means based on your identified parameters

data_pivot = df_ref1.pivot_table(
    index=['age_group', 'education'],
    values='days_employed',
    aggfunc='mean',
)

data_pivot

days_employed
age_group education                         
18-29     bachelor's degree      1225.819806
          primary education      1236.562385
          secondary education    1228.268849
          some college           1045.216936
30-39     bachelor's degree      2020.202275
          graduate degree        4159.753909
          primary education      1711.383434
          secondary education    2062.472022
          some college           1562.183882
40-49     bachelor's degree      2719.149072
          graduate degree         409.200149
          primary education      1793.631404
          secondary education    2766.541153
          some college           2451.018995
50-59     bachelor's degree      3306.590162
          graduate degree        5352.038180
          primary education      3132.947220
          secondary education    3259.390951
          some college           2665.876064
60-69     bachelor's degree      7842.726726
          graduate degree        8795.549101
          primary education      8411.921496
          secondary education    8245.987859
          some college           7512.882571
70+       bachelor's degree      8946.682133
          primary education      8795.549101
          secondary education    8048.691910
          some college           8795.549101

As we can see in pivot tables mean values are greater than median values. That means the data are skewed to the left, and we have to use the median to replace missing values using `filling_missing_values` function with correct domains.

In [181]:
# Apply function to the days_employed

column='days_employed'
factors=['age_group', 'education']
function= 'median'
filling_missing_values(column, factors, function)

In [182]:
# Check if function worked

df[df['days_employed'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group


In [183]:
# Check the entries in all columns - make sure we fixed all missing values

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
age_group           21525 non-null object
dtypes: float64(2), int64(5), object(6)
memory usage: 2.1+ MB


### Checking duplicates

Find the number of obvious duplicates in the table. If they are exist call the `pandas` method for getting rid of obvious duplicates, then count obvious duplicates once more to make duplicates have been removed.

In [184]:
# Checking duplicates

print(df.duplicated().sum()) # counting clear duplicates
print()
print(df.duplicated().sum()/df.shape[0]*100) # counting the percentage of clear duplicates

147

0.6829268292682927


In [185]:
# Address the duplicates, if they exist

df = df.drop_duplicates() # removing obvious duplicates

In [186]:
# Last check whether we have any duplicates

print(df.duplicated().sum())

0


In [187]:
# Check the size of the dataset that you now have after your first manipulations with it

df.info()
print()
print(df.isna().sum()/df.shape[0]*100) #check the percentage of missing values

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21378 entries, 0 to 21524
Data columns (total 13 columns):
children            21378 non-null int64
days_employed       21378 non-null float64
dob_years           21378 non-null int64
education           21378 non-null object
education_id        21378 non-null int64
family_status       21378 non-null object
family_status_id    21378 non-null int64
gender              21378 non-null object
income_type         21378 non-null object
debt                21378 non-null int64
total_income        21378 non-null float64
purpose             21378 non-null object
age_group           21378 non-null object
dtypes: float64(2), int64(5), object(6)
memory usage: 2.3+ MB

children            0.0
days_employed       0.0
dob_years           0.0
education           0.0
education_id        0.0
family_status       0.0
family_status_id    0.0
gender              0.0
income_type         0.0
debt                0.0
total_income        0.0
purpose             0

The amount of clear duplicates compared to the whole dataset is less than 1%. 

## **Categorization of data**

To answer the questions and test the hypotheses, we have to work with categorized data. Three columns should be categorized: `'children'`, `'total_income'`, and `'purpose'`.


1. The `'children_id'` column data needs to be turned into following categories:
    - `'no_kids'`  - client doesn't have any kids
    - `'has kids'`  - client has one or more kids

2. The `'purpose_category'` column data needs to be turned into following categories:
    - `'real estate'`
    - `'vehile'`
    - `'wedding'`
    - `'education'`

3. The `'income_level'` column data contains following categories:
    - `'low'`  - below 25% quantile
    - `'medium'`  - from 25% to 75% quantile
    - `'high'`  - above 75% quantile

For `'children'` categorization we should:
- Check the unique values in `'children'` column
- Create the `'assign_children_id'` function to assign clients for two categories: clients who have kids and clients who don't have kids, because exact kids quantity doesn't matter for further investigation
- create a new `'children_id'` column that contains new categories.

In [188]:
# Check the unique values in 'children' column

print(df.children.unique())

[1 0 3 2 4 5]


In [189]:
# Let's write a function to categorize the data in 'children' column

def assign_children_id(children):
    if children > 0:
        return 'has kids' #has kids
    else: 
        return 'no kids' #no kids

In [190]:
# Test if the function works

#Test function on 2 (should be 'has kids')
print(assign_children_id(2))
print()
#Test function on 0 (should be 'no kids')
print(assign_children_id(0))
print()
#Test function on 5 (should be 'has kids')
print(assign_children_id(5))

has kids

no kids

has kids


In [191]:
# Creating new column based on function

df['children_id']=df['children'].apply(assign_children_id)
df['children_id'].value_counts()

no kids     14015
has kids     7363
Name: children_id, dtype: int64

For `'total_income'` categorization we should:
- generate descriptive statistics to get exact numbers for 25, 50, and 70 percent quantile.
- create `assign_income_lvl` function to assign clients for three categories: 
    - `'low'`, 
    - `'medium'`, and
    - `'high'`.
- create `income_lvl` column with new categories

In [192]:
#Generate descriptive statistics

df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21378.000000,21378.000000,21378.000000,21378.000000,21378.000000,21378.000000,21378.000000
mean,0.478716,3455.756000,44.756245,0.816540,0.975021,0.081299,26499.994040
std,0.752063,3190.377617,13.923677,0.549486,1.422536,0.273299,15750.680051
min,0.000000,24.141633,19.000000,0.000000,0.000000,0.000000,3306.762000
25%,0.000000,1005.389899,33.000000,1.000000,0.000000,0.000000,17166.389500
50%,0.000000,2053.869341,43.000000,1.000000,0.000000,0.000000,23022.711000
75%,1.000000,5335.833813,55.000000,1.000000,1.000000,0.000000,31714.888000
max,5.000000,17615.563266,75.000000,4.000000,4.000000,1.000000,362496.645000


In [193]:
# Let's write a function to categorize the data in 'total_income' column

def assign_income_lvl(total_income):
    
    if total_income < df['total_income'].quantile(0.25):
        return 'low' #low income
    elif total_income > df['total_income'].quantile(0.75):
        return 'high' #high income
    else: 
        return 'medium' #medium income

In [194]:
# Test if the function works

#Test function on 50000 (should be 'high')
print(assign_income_lvl(50000))
print()
#Test function on 15000 (should be 'low')
print(assign_income_lvl(15000))
print()
#Test function on 29000 (should be 'medium')
print(assign_income_lvl(29000))

high

low

medium


In [ ]:
# Creating new column based on function

df['income_lvl']=df['total_income'].apply(assign_income_lvl)
df['income_lvl'].value_counts()

For `'purpose'` categorization we should:
- check the unique values in `'purpose'` column
- import all libraries and packages for lemmatization 
- create list of words for all unique values in purposes column and count each unique value
- create list of keywords for each category
- create the `'lemmatization_func'` function to assign clients for four categories: 
    - `'real estate'`
    - `'vehile'`
    - `'wedding'`
    - `'education'`
- create a new `'purpose_category'` column that contains new categories.

In [ ]:
print(df['purpose'].sort_values().unique())

In [ ]:
#import all libraries and packages for lemmatization 

from pymystem3 import Mystem
from collections import Counter
m = Mystem()
# importing WordNet Lemmatizer:
import nltk
from nltk.stem import WordNetLemmatizer
wordnet_lemma = WordNetLemmatizer()
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

In [ ]:
lemmas_list_all = []

for purpose in df['purpose'].unique():
    words = nltk.word_tokenize(purpose)
    lemmas = [wordnet_lemma.lemmatize(w, pos = 'n') for w in words]
    for i in lemmas:
        lemmas_list_all.append(i)
        
Counter(lemmas_list_all)

In [ ]:
#create categories with keywords

real_estate_category=['house','housing','estate','property','building']
vehile_category=['car']
wedding_category=['wedding','ceremony']
education_category=['education','educated','university']

In [ ]:
# Let's write a function to categorize the data based on common topics
    
def lemmatization_func(purpose):
  
    words = nltk.word_tokenize(purpose)
    lemmas = [wordnet_lemma.lemmatize(w, pos = 'n') for w in words]
    
    if any(word in lemmas for word in real_estate_category):
        return 'real estate'
    elif  any(word in lemmas for word in vehile_category):
        return 'vehile'
    elif  any(word in lemmas for word in wedding_category):
        return 'wedding'
    elif  any(word in lemmas for word in education_category):
        return 'education'
    else:
        
        return 'other'


In [ ]:
# Create a column with the categories and count the values for them

df['purpose_category']=df['purpose'].apply(lemmatization_func)
df['purpose_category'].value_counts()

## **Checking the Hypotheses**


### Is there a correlation between having children and paying back on time?

In [ ]:
# Check the children data and paying back on time

print('Distribution of clients with and without kids')
print()
print(df['children_id'].value_counts(dropna=False)/df.shape[0]*100)
print()
print('Distribution of clients who paid off the loan and who have a debt.')
print()
print(df['debt'].value_counts(dropna=False)/df.shape[0]*100)
print()

# Calculating default-rate based on the number of children

print('Default-rate based on the having children')
print()
print(df.groupby('children_id')['debt'].apply(lambda x: x.sum()/x.count()*100).round(1).sort_values())

#in case if we need default-rate based on the exact number of children
#print(df.groupby('children')['debt'].apply(lambda x: x.sum()/x.count()*100).round(1))

**Conclusion**

The data reveal differences in default rate:
- People without kids more often don't have any debts than people with kids. However, the difference is less than 2% and the representation of both groups is not equal, which allows us to neglect those data and conclude lack of evidence dependence default-rate on the number of children.

### Is there a correlation between family status and paying back on time?

In [ ]:
# Check the family status data and paying back on time

print('Distribution of clients with different marital status')
print()
print(df['family_status'].value_counts(dropna=False)/df.shape[0]*100)
print()

# Calculating default-rate based on family status

print('Calculating default-rate based on family status')
print()
print(df.groupby('family_status')['debt'].apply(lambda x: x.sum()/x.count()*100).round(1).sort_values())


**Conclusion**

The data reveal differences in default rate:

- We can see slight linear dependence between different marital statuses and default rates. The widow/widower marital status group has the lowest default rate the unmarried marital status group has the highest default rate. The difference between the maximum and minimum default rates is 3.2%. However, representation of Widow/widower marital status group is less than 5%, and representation of an unmarried marital status group is 13%, which allows us to neglect those data and conclude lack of evidence dependence default-rate on the marital status.

### Is there a correlation between income level and paying back on time?

In [ ]:
# Check the income level data and paying back on time

print('Distribution of clients with different income level')
print()
print(df['income_lvl'].value_counts(dropna=False)/df.shape[0]*100)
print()

# Calculating default-rate based on income level

print('Calculating default-rate based on income level')
print()
print(df.groupby('income_lvl')['debt'].apply(lambda x: x.sum()/x.count()*100).round(1).sort_values())

**Conclusion**

The data reveal differences in default rate:

- We can see almost no linear dependency between different income levels and default rates. Clients with high-income levels have the lowest default rate, clients with medium income levels have the highest default rate. The difference between the maximum and minimum default rates is 0.2%. However, the difference is so small which allows us to neglect those data and conclude lack of evidence dependence default-rate on the income level.

### How does credit purpose affect the default rate?

In [ ]:
# Check the income level data and paying back on time

print('Distribution of clients with different credit purpose')
print()
print(df['purpose_category'].value_counts(dropna=False)/df.shape[0]*100)
print()

# Check the percentages for default rate for each credit purpose and analyze them

print('Calculating default-rate based on credit purpose')
print()
print(df.groupby('purpose_category')['debt'].apply(lambda x: x.sum()/x.count()*100).round(1).sort_values())

**Conclusion**

The data reveal differences in default rate:

- We can see slight linear dependence between purpose category and default rates. The real estate purpose group has the lowest default rate, the vehile purpose group has the highest default rate. The difference between the maximum and minimum default rates is 2.1%. However, representation of groups is not equal, which allows us to neglect those data and conclude lack of evidence dependence default-rate on the credit purpose.

## **General Conclusion** 

My project was a report prepared for a bank’s loan division. We needed to find out if a customer’s marital status, income level, loan purposes, and the number of children have an impact on whether they will default on a loan.

Original Data Frame contained:
- Different registres
- Incorrect artifacts
- Incorrect data types
- Negative numbers
- Over 10% of quantitative missing values in `'total_income'` and `'days_employed'` columns

During the **` Data exploration`** process we didn't find any patterns in distribution. Based on this, we concluded that missing values are random for both columns. 

Then, during the **` Data transformation`** process we fixed all artifacts, registers, and incorrect data types which happened due to technical issues. That task prepared our Data Frame for missing values restoration.

During **`Missing values restoration`** process we categorized the client's age into 6 age groups:
- `18-29`
- `30-39`
- `40-49` 
- `50-59`
- `60-69`
- `70+`  
For the `'total_income'` column we checked mean and median values for small groups with a unique combination of `age group`, `education`, and `income type`. As we saw in pivot tables mean values were greater than median values. That means the data are skewed to the left, and missing values should be replaced with median values.

For the `'days_employed'` column we checked mean and median values for small groups with a unique combination of `age group` and `education`. As we saw in pivot tables mean values were greater than median values. That means the data are skewed to the left, and missing values should be replaced with median values.

After the restoration process, our Data Frame was ready to be used in our investigation. 

We had 4 questions:
- Is there a connection between having kids and repaying a loan on time?
- Is there a connection between marital status and repaying a loan on time?
- Is there a connection between income level and repaying a loan on time?
- How do different loan purposes affect on-time loan repayment?
The best way to answer those questions was data categorization. That lets us get the most accurate data representation.
During **`Data categorisation`** process we've got 3 new columns:
1. The `'children_id'` column contains following categories:
    - `'no_kids'`  - client doesn't have any kids
    - `'has kids'`  - client has one or more kids

2. The `'purpose_category'` column data contains following categories:
    - `'real estate'`
    - `'vehile'`
    - `'wedding'`
    - `'education'`

3. The `'income_level'` column data contains following categories:
    - `'low'`  - below 25% quantile
    - `'medium'`  - from 25% to 75% quantile
    - `'high'`  - above 75% quantile

During our final investigation we've got 4 important conclusions:
- People without kids more often don't have any debts than people with kids. However, the difference is less than 2% and the representation of both groups is not equal, which allows us to neglect those data and conclude lack of evidence dependence default-rate on the number of children.
- We can see slight linear dependence between different marital statuses and default rates. The widow/widower marital status group has the lowest default rate the unmarried marital status group has the highest default rate. The difference between the maximum and minimum default rates is 3.2%. However, representation of Widow/widower marital status group is less than 5%, and representation of an unmarried marital status group is 13%, which allows us to neglect those data and conclude lack of evidence dependence default-rate on the marital status.
- We can see almost no linear dependency between different income levels and default rates. Clients with high-income levels have the lowest default rate, clients with medium income levels have the highest default rate. The difference between the maximum and minimum default rates is 0.2%. However, the difference is so small which allows us to neglect those data and conclude lack of evidence dependence default-rate on the income level.
- We can see slight linear dependence between purpose category and default rates. The real estate purpose group has the lowest default rate, the vehicle purpose group has the highest default rate. The difference between the maximum and minimum default rates is 2.1%. However, the representation of groups is not equal, which allows us to neglect those data and conclude lack of evidence dependence default-rate on the credit purpose.